<center>
    <h1> INF491 - Astroinformatics </h1>
    <h2> WavClumps Project:  A Wavelet based algorithm to find clumps and its structure </h2>
    <h3> Martín Villanueva A. <sup>1</sup></h3>
</center>

[1] _martin.villanueva@alumnos.usm.cl - DI UTFSM - June 2016._

## Table of Contents
* [Introduction](#intro)
* [Proposed Solution](#proposal)
* [Concept Test and Experiments](#tests)

<div id='intro' />
## Introduction

In concise words the problem to be addressed consists on given a 3D spectroscopic cube of data (mostly from observations of cold molecular clouds), build an algorithm that automatically can identify clump structures, and determine the hierarchical relationships between them. **Why?**

* Manual methods **doesn’t scale** as fast as the generation of new data observations.
* Size of the images is **huge** and could contain hundreds/thousands of cores.
* Clump identification performed (manually) by astronomers usually didn’t match (**biased methods**).
* Determine the hierarchical relationships between the found clumps is fundamental to understand processes like the **star formation**.
* Clumping algorithms **(GaussClump, ClumpFind, FellWalker)** do not face the problem of finding hierarchical structures.
* State of the Art algorithm that that face this problem (Gregorio et al.) have only worked with **2D data** (collapsed cubes).

<div id='proposal' />
## Proposed Solution

The proposed approach is the natural extension of Gregorio et al. to 3D spectroscopic data cubes. The solution outline of the methods is shown in the next image:
<img src="files/wavclump.png" style="width: 500px;">
As you can see, it corresponds to a **multi-stage** algorithm with the following steps:

1. *Multiresolution Analysis* (**MRA**) is performed through a 3D Discrete Wavelet Transform (**3D-DWT**), to extract the features at different scales/frequencies.
2. For each computed cube approximation a 3D clump *identification/segmentation* algorithm is applied, in order to find structures on each level.
3. The results of each level will be (in some way) linked, according to his hierarchical structure. This will be summarized in a *tree structure* like a **Dendrogram**

### 1. Multiresolution Analysis through 3D-DWT

MRA is achieved by computing the 3D-DWT for the data cube at different levels, and then reconstructing the image with the approximation coefficients obtained at each level.

**1) 1D-DWT.** Starting from a signal s of length N, two sets of coefficients are computed: approximation coefficients CA1, and detail coefficients CD1. These vectors are obtained by convolving s with the low-pass filter Lo_D for approximation and with the high-pass filter Hi_D for detail, followed by *dyadic decimation*.
<img src="files/1Ddwt.gif" style="width: 500px;">

**2) 2D-DWT.** This two-dimensional DWT leads to a decomposition of approximation coefficients at level j in four components: the _approximation_ at level j + 1, and the details in three orientations (_horizontal_, _vertical_, and _diagonal_). The following chart describes the basic decomposition steps for images:
<img src="files/2Ddwt.gif" style="width: 500px;">

### 2. Clump indentification/segmentation

To perform this step two classic algorithms will be used:
1. **ClumpFind.** The algorithm works by contouring the data at multiples of the rms noise of the observations. It starts from the highest level, where isolate cores are identified and each of these is considered as new clump. Then this step is repeated by gradually decreasing the level where to search, identifying isolate structures and connecting them with clumps identified at previous levels, or defining them as new clumps otherwise.
<img src="files/cf.png" style="width: 300px;">

2. **FellWalker.** The algorithms works by computing ascent paths (with the highest gradient) for each single pixel. These paths have two alternatives: 1) Reach a local peak and so find a new clump. 2) Reach another ascent path computed previously, and so assign the pixels on that path to the corresponding (crashed) clump.
<img src="files/fw.png" style="width: 300px;">

**Observation.** An important feature must be taken into account: At higher levels (and lower frequencies) the resulting images are **much more easy/trivial to segmentate!**. Let's see for example, the MRA performed over a FITS image of Orion Methanol observations (slices at frequencies are shown).

**Level 0** (no MRA)
<img src="files/level0.png" style="width: 800px;">
**Level 1**
<img src="files/level1.png" style="width: 800px;">
**Level 2**
<img src="files/level2.png" style="width: 800px;">
**Level 3**
<img src="files/level3.png" style="width: 800px;">
it is clear that in the high levels the clump structures are much more easy to identify that the ones in lower levels. **Then simpler algorithms could be used!** as those of *computer vision/image processing*: **Simple Thresholding**, **Watershed algorithm**, etc. The only **complication** is that (some of them) must be **adapted** for 3D data.

### 3. Hierarchical relations
In order to determine how clumps at different levels are related to each other, the following methods could be used:
1. If the max pixel of a clump in the level $i$ it's contained on a clump at level $i+1$, then we say we say that clump al level $i$ proceeds from the corresponding clump at level $i+1$.
2. The other alternative is to find the **center of mass** of clump at level $i$, and similarly if it is contained on a clump at level $i+1$, then the relationship is inferred.
The results of such process are summarized in a tree-like structure (Dendrogram).

<div id='tests' />
## Concept Test and Experiments

In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matlab.engine

sys.path.append('../../ACALIB/')
from acalib import acontainer as ac
from acalib.io import graph as gp


sys.path.insert(0, './clumping')
import fellwalker as fwalker

### Matlab engine to use

In [2]:
eng = matlab.engine.start_matlab()

### Testing some wavelets

In first place let's see the efects of applying 3D-DWT on a FITS image to perform multiresolution analysis, with different Wavelets families. 

In [3]:
eng.wavemenu(nargout=0)

### Loading FITS data

In [4]:
"""
Loading data
"""
#fits_path = '../../bindata/fits/cubes/Antennae_North.CO3_2Line.Clean.pcal1.image.fits'
#fits_path = '../../bindata/fits/cubes/Antennae_South.CO3_2Line.Clean.pcal1.image.fits'
#fits_path = '../../bindata/fits/cubes/CenA.CO2_1Line.Clean.image.fits'
#fits_path = '../../bindata/fits/cubes/M100line.image.fits'
fits_path = '../../bindata/fits/cubes/Orion.methanol.cbc.contsub.image.fits'

if os.path.isfile(fits_path):
    ml_data = eng.load_fits(fits_path)
    np_data = np.asarray(ml_data)
    np_data = ma.masked_array(np_data, mask=np.isnan(np_data))
else: print("FITS file doesn't exist")

In [ ]:
A,D = eng.mra(ml_data, 'sym5', 3, nargout=2)

In [ ]:
fw = fwalker.FellWalker()
caa,clump = fw.run(np_data, verbose=True)

INFO: CAA initialized successfully [fellwalker]
INFO: [Stage 1] Removing small isolated regions [fellwalker]
INFO: [Stage 2] Walking up! [fellwalker]


In [ ]:
class WavClumps():
    def __init__(self, wavelet='sym5', nlevel=4, clumping_method='fellwalker'):
        self.wavelet = wavelet
        self.nlevel = nlevel
        #setting the clumping algorithm
        if clumping_method=='fellwalker':
            self.clumping = fwalker.FellWalker()
        elif clumping_method=='clumpfind':
            pass
        #setting default params
        self.default_params()
    
    def default_params(self):
        pass
    
    def mra(self, ml_data):
        """
        Computes approximations and details for each level
        """
        _A,_D = eng.mra(ml_data, self.wavelet, self.nlevel, nargout=2)
        
        #casting to numpy.ndarray
        A = map(np.asarray, _A)
        D = map(np.asarray, _D)
        del _A,_D
        return (A,D)
    
    def dendro_build(self):
        pass
    
    def run(self, np_data, ml_data):
        #perform multiresolution analysis
        A,D = self.mra(ml_data)
        A.insert(0, np_data)
        
        #perform clumping algorithm over original data and each approximation level
        caa = list()
        clumps = list()
        for data in A:
            res = self.clumping.run(data)
            caa.append(res[0])
            clumps.append(res[1])
        
        #storing results
        self.caa = caa
        self.clumps = clumps
        return caa,clumps
    
    def summarize(self):
        for i in range(self.nlevel+1):
            print('Level {0}:'.format(i))
            print('Number of clumps: {0}'.format(len(self.clumps[i])))
            mval = 0
            for pixels in self.clumps[i].values():
                npix = len(pixels)
                if npix>mval: mval = npix
            print('Number of pixels of bigger clump:{0}'.format(mval))
            print('------------------------------------------')

### Instances with differente Wavelet family 

In [ ]:
wc1 = WavClumps(wavelet='haar', nlevel=4)
wc2 = WavClumps(wavelet='db2', nlevel=4)
wc3 = WavClumps(wavelet='sym3', nlevel=4)
wc4 = WavClumps(wavelet='bior4.4', nlevel=4)
wc5 = WavClumps(wavelet='rbio4.4', nlevel=4)

In [ ]:
caa1, clumps1 = wc1.run(np_data, ml_data)
caa2, clumps2 = wc2.run(np_data, ml_data)
caa3, clumps3 = wc3.run(np_data, ml_data)
caa4, clumps4 = wc4.run(np_data, ml_data)
caa5, clumps5 = wc5.run(np_data, ml_data)

In [ ]:
"""
Dictionary to handle results
"""
caa = {'haar':caa1, 'db2':caa2, 'sym3':caa3, 'bior4.4':caa4, 'rbio4.4':caa5}

### Some sumarizing results

In [ ]:
"""
Haar wavelet
"""
wc1.summarize()

In [ ]:
"""
Daubechies wavelet
"""
wc2.summarize()

In [ ]:
"""
Near symmetric wavelet
"""
wc3.summarize()

In [ ]:
"""
Biorthogonal wavelet
"""
wc4.summarize()

In [ ]:
"""
Reverse biorthogonal wavelet
"""
wc5.summarize()

### Some 2D graphical results

In [ ]:
def show_caa_slices(caa, level, dslc=10):
    caa = np.copy(caa)
    caa[caa==-1] = 0
    maxf = caa.shape[2]
    idx_slices = np.linspace(dslc,maxf-dslc,9, dtype=np.int)
    plt.figure(figsize=(15,15))
    for i in range(9):
        f = idx_slices[i]
        plt.subplot(3,3,i+1)
        plt.imshow(caa[:,:,f])
        plt.xlabel('Freq = {0}'.format(f))
        if i==1:
            plt.title('CAA slices at level {0}'.format(level))
    plt.show()

In [ ]:
level = 1
show_caa_slices(caa['bior4.4'][level], level)

### Some 3D graphical results

In [ ]:
""""
Cube where to load CAAs
"""
cont=ac.AContainer()
cont.load('../../bindata/fits/cubes/Orion.methanol.cbc.contsub.image.fits')
cube=cont.primary.copy()

In [ ]:
level = 2
cube.data = caa['haar'][level]
gp.volume(cube)